In [1]:
import os
import gzip
import shutil

# Define the directory containing the files
input_dir = "GSE235063_RAW"  # Replace with the actual folder path
output_dir = os.path.join(input_dir, "processed_data_csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# File patterns to look for
file_patterns = ["processed_barcodes.tsv", "processed_genes.tsv", "processed_metadata.tsv", "processed_matrix.mtx"]

# Loop through files in the input directory
for filename in os.listdir(input_dir):
    # Check if the file matches any of the desired patterns
    if any(pattern in filename for pattern in file_patterns):
        file_path = os.path.join(input_dir, filename)
        output_file_path = os.path.join(output_dir, filename.replace(".tsv", ".csv").replace(".mtx", ".csv"))

        # Extract and convert gzipped files if necessary
        if filename.endswith(".gz"):
            with gzip.open(file_path, 'rt') as gz_file, open(output_file_path, 'w') as out_file:
                shutil.copyfileobj(gz_file, out_file)
        else:
            # For uncompressed files, copy them to the new location with a .csv extension
            shutil.copyfile(file_path, output_file_path)

print(f"Processed data has been extracted and saved to {output_dir}")


Processed data has been extracted and saved to GSE235063_RAW\processed_data_csv


In [ ]:
import os
import gzip
from scipy.io import mmread
import pandas as pd

# Define input and output directories
input_dir = "GSE235063_RAW"  # Replace with your directory path
output_dir = os.path.join(input_dir, "processed_matrices_csv")
os.makedirs(output_dir, exist_ok=True)

# Process only the *_processed_matrix.mtx.gz files
for filename in os.listdir(input_dir):
    if filename.endswith("_processed_matrix.mtx.gz"):
        # Define file paths
        gz_file_path = os.path.join(input_dir, filename)
        decompressed_path = gz_file_path.replace(".gz", "")
        output_csv_path = os.path.join(output_dir, filename.replace(".mtx.gz", ".csv"))

        try:
            # Decompress the .gz file
            with gzip.open(gz_file_path, 'rt') as gz_file:
                with open(decompressed_path, 'w') as decompressed_file:
                    decompressed_file.write(gz_file.read())

            # Read the sparse matrix
            sparse_matrix = mmread(decompressed_path)
            # Convert to a dense DataFrame
            dense_matrix = pd.DataFrame(sparse_matrix.toarray())
            # Save the dense matrix as a CSV
            dense_matrix.to_csv(output_csv_path, index=False)
            print(f"Processed matrix saved: {output_csv_path}")

            # Clean up: Remove the temporary decompressed file
            os.remove(decompressed_path)

        except Exception as e:
            print(f"Error processing {gz_file_path}: {e}")

print(f"All processed matrices have been saved to {output_dir}")

In [14]:
import os
import pandas as pd

# Define the directory paths
processed_matrices_dir = "GSE235063_RAW/processed_matrices_csv"
genes_dir = "GSE235063_RAW"  # Directory containing *_processed_genes.tsv files

# Iterate through all processed matrix files
for matrix_file in os.listdir(processed_matrices_dir):
    if matrix_file.endswith("_processed_matrix.csv"):
        # Define file paths
        matrix_path = os.path.join(processed_matrices_dir, matrix_file)
        gene_file_name = matrix_file.replace("_processed_matrix.csv", "_processed_genes.tsv.gz")
        gene_file_path = os.path.join(genes_dir, gene_file_name)
        
        # Load the matrix file
        matrix_df = pd.read_csv(matrix_path)
        
        # Load the corresponding gene file
        if os.path.exists(gene_file_path):
            genes_df = pd.read_csv(gene_file_path, header=None, names=["Gene"])  # Assume no header in TSV
        else:
            print(f"Gene file missing for {matrix_file}")
            continue
        
        # Add the "Gene" column to the matrix
        matrix_df.insert(0, "Gene", genes_df["Gene"])
        
        # Rename all other column headers to "AML"
        matrix_df.columns = ["Gene"] + ["AML"] * (len(matrix_df.columns) - 1)
        
        # Save the updated matrix back to the same file
        matrix_df.to_csv(matrix_path, index=False)
        print(f"Updated matrix file: {matrix_path}")

print("All files updated.")


Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494257_AML16_DX_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494258_AML16_REL_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494259_AML16_REM_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494260_AML6_DX_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494261_AML6_REL_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494262_AML6_REM_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494263_AML2_DX_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494264_AML2_REL_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494265_AML2_REM_processed_matrix.csv
Updated matrix file: GSE235063_RAW/processed_matrices_csv\GSM7494266_AML15_DX_processed_matrix.csv
Updated ma

In [ ]:
###LSTM
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Directories
processed_matrices_dir = "GSE235063_RAW/processed_matrices_csv"
output_dir = "results/lstm_bdm_analysisV2"
os.makedirs(output_dir, exist_ok=True)

# Train LSTM and Extract Feature Importance
def train_lstm_and_extract_importance(sequences, labels, gene_names, patient):
    # Normalize data
    scaler = MinMaxScaler()
    sequences = scaler.fit_transform(sequences)

    # Reshape to (samples, timesteps, features)
    sequences = sequences.reshape((sequences.shape[0], sequences.shape[1], 1))

    # Generate dummy labels matching samples (one per row/gene)
    labels = np.random.randint(0, 3, size=sequences.shape[0])  # Placeholder for actual labels

    # 70:15:15 split (train, validation, test)
    split_1 = int(0.7 * len(sequences))
    split_2 = int(0.85 * len(sequences))
    X_train, X_val, X_test = (
        sequences[:split_1],
        sequences[split_1:split_2],
        sequences[split_2:],
    )
    y_train, y_val, y_test = labels[:split_1], labels[split_1:split_2], labels[split_2:]

    # Build LSTM model
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),
        Dense(3, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=16,
        verbose=1
    )

    # Save training metrics
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(output_dir, f"{patient}_training_metrics.csv"), index=False)

    # Evaluate on test data
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=["DX", "REL", "REM"])
    confusion = confusion_matrix(y_test, y_pred)

    # Save evaluation metrics
    with open(os.path.join(output_dir, f"{patient}_evaluation_metrics.txt"), "w") as f:
        f.write(f"Accuracy: {accuracy}\n")
        f.write(f"Classification Report:\n{report}\n")
        f.write(f"Confusion Matrix:\n{confusion}\n")

    # Extract feature importance from LSTM weights
    lstm_weights = model.layers[0].get_weights()[0]  # LSTM input weights
    feature_importance = np.mean(np.abs(lstm_weights), axis=(0, 1))  # Average importance across gates

    # Rank features by importance
    feature_ranking = pd.DataFrame({
        "Gene": gene_names,
        "Importance": feature_importance
    }).sort_values(by="Importance", ascending=False)

    return feature_ranking

# Process one patient at a time
def process_patient(patient, files):
    print(f"Processing {patient}...")

    # Load DX, REL, REM data
    dx_file = os.path.join(processed_matrices_dir, files.get("DX"))
    rel_file = os.path.join(processed_matrices_dir, files.get("REL"))
    rem_file = os.path.join(processed_matrices_dir, files.get("REM"))

    dx_df = pd.read_csv(dx_file)
    rel_df = pd.read_csv(rel_file)
    rem_df = pd.read_csv(rem_file)

    # Extract genes and samples
    common_genes = set(dx_df["Gene"]).intersection(rel_df["Gene"]).intersection(rem_df["Gene"])
    dx_df = dx_df[dx_df["Gene"].isin(common_genes)].set_index("Gene")
    rel_df = rel_df[rel_df["Gene"].isin(common_genes)].set_index("Gene")
    rem_df = rem_df[rem_df["Gene"].isin(common_genes)].set_index("Gene")

    # Combine DX, REL, REM into sequences (samples x timepoints)
    sequences = np.stack([dx_df.mean(axis=1).values, rel_df.mean(axis=1).values, rem_df.mean(axis=1).values], axis=1)
    labels = np.array([0, 1, 2])  # DX=0, REL=1, REM=2

    # Train LSTM and extract feature importance
    feature_ranking = train_lstm_and_extract_importance(sequences, labels, dx_df.index.tolist(), patient)

    # Save top 100 features
    top_100_features = feature_ranking.head(100)
    top_100_features.to_csv(os.path.join(output_dir, f"{patient}_top_100_features.csv"), index=False)

    print(f"Completed {patient}")

# Organize files by patient
file_groups = {}
for file in os.listdir(processed_matrices_dir):
    if not file.endswith("_processed_matrix.csv"):
        continue
    parts = file.split("_")
    patient, state = parts[1], parts[2]
    file_groups.setdefault(patient, {})[state] = file

# Run the analysis
for patient, files in file_groups.items():
    process_patient(patient, files)

print("LSTM feature importance analysis completed.")

In [ ]:
##Bidirectional LSTM

In [18]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Directories
processed_matrices_dir = "GSE235063_RAW/processed_matrices_csv"
output_dir = "results/bidirectional_lstm_analysis"
os.makedirs(output_dir, exist_ok=True)

# Train Bidirectional LSTM and Extract Feature Importance
def train_bilstm_and_extract_importance(sequences, labels, gene_names, patient):
    # Normalize data
    scaler = MinMaxScaler()
    sequences = scaler.fit_transform(sequences)

    # Reshape to (samples, timesteps, features)
    sequences = sequences.reshape((sequences.shape[0], sequences.shape[1], 1))

    # Generate dummy labels matching samples (one per row/gene)
    labels = np.random.randint(0, 3, size=sequences.shape[0])  # Placeholder for actual labels

    # 70:15:15 split (train, validation, test)
    split_1 = int(0.7 * len(sequences))
    split_2 = int(0.85 * len(sequences))
    X_train, X_val, X_test = (
        sequences[:split_1],
        sequences[split_1:split_2],
        sequences[split_2:],
    )
    y_train, y_val, y_test = labels[:split_1], labels[split_1:split_2], labels[split_2:]

    # Build Bidirectional LSTM model
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))),
        Dropout(0.3),
        Bidirectional(LSTM(32, return_sequences=False)),
        Dense(32, activation="relu"),
        Dropout(0.2),
        Dense(3, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=16,
        verbose=1
    )

    # Save training metrics
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(output_dir, f"{patient}_training_metrics.csv"), index=False)

    # Evaluate on test data
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=["DX", "REL", "REM"])
    confusion = confusion_matrix(y_test, y_pred)

    # Save evaluation metrics
    with open(os.path.join(output_dir, f"{patient}_evaluation_metrics.txt"), "w") as f:
        f.write(f"Accuracy: {accuracy}\n")
        f.write(f"Classification Report:\n{report}\n")
        f.write(f"Confusion Matrix:\n{confusion}\n")

    # Extract feature importance from LSTM weights
    lstm_weights = model.layers[0].get_weights()[0]  # Input weights of the first LSTM layer
    feature_importance = np.mean(np.abs(lstm_weights), axis=(0, 1))  # Average importance across gates

    # Rank features by importance
    feature_ranking = pd.DataFrame({
        "Gene": gene_names,
        "Importance": feature_importance
    }).sort_values(by="Importance", ascending=False)

    return feature_ranking

# Process one patient at a time
def process_patient(patient, files):
    print(f"Processing {patient}...")

    # Load DX, REL, REM data
    dx_file = os.path.join(processed_matrices_dir, files.get("DX"))
    rel_file = os.path.join(processed_matrices_dir, files.get("REL"))
    rem_file = os.path.join(processed_matrices_dir, files.get("REM"))

    dx_df = pd.read_csv(dx_file)
    rel_df = pd.read_csv(rel_file)
    rem_df = pd.read_csv(rem_file)

    # Extract genes and samples
    common_genes = set(dx_df["Gene"]).intersection(rel_df["Gene"]).intersection(rem_df["Gene"])
    dx_df = dx_df[dx_df["Gene"].isin(common_genes)].set_index("Gene")
    rel_df = rel_df[rel_df["Gene"].isin(common_genes)].set_index("Gene")
    rem_df = rem_df[rem_df["Gene"].isin(common_genes)].set_index("Gene")

    # Combine DX, REL, REM into sequences (samples x timepoints)
    sequences = np.stack([dx_df.mean(axis=1).values, rel_df.mean(axis=1).values, rem_df.mean(axis=1).values], axis=1)
    labels = np.array([0, 1, 2])  # DX=0, REL=1, REM=2

    # Train Bidirectional LSTM and extract feature importance
    feature_ranking = train_bilstm_and_extract_importance(sequences, labels, dx_df.index.tolist(), patient)

    # Save top 100 features
    top_100_features = feature_ranking.head(100)
    top_100_features.to_csv(os.path.join(output_dir, f"{patient}_top_100_features.csv"), index=False)

    print(f"Completed {patient}")

# Organize files by patient
file_groups = {}
for file in os.listdir(processed_matrices_dir):
    if not file.endswith("_processed_matrix.csv"):
        continue
    parts = file.split("_")
    patient, state = parts[1], parts[2]
    file_groups.setdefault(patient, {})[state] = file

# Run the analysis
for patient, files in file_groups.items():
    process_patient(patient, files)

print("Bidirectional LSTM feature importance analysis completed.")


Processing AML16...
Epoch 1/50
1065/1065 [==============================] - 77s 55ms/step - loss: 1.0987 - accuracy: 0.3329 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 2/50
1065/1065 [==============================] - 46s 43ms/step - loss: 1.0989 - accuracy: 0.3360 - val_loss: 1.0989 - val_accuracy: 0.3328
Epoch 3/50
1065/1065 [==============================] - 47s 44ms/step - loss: 1.0987 - accuracy: 0.3387 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 4/50
1065/1065 [==============================] - 43s 41ms/step - loss: 1.0986 - accuracy: 0.3378 - val_loss: 1.0987 - val_accuracy: 0.3339
Epoch 5/50
1065/1065 [==============================] - 41s 38ms/step - loss: 1.0987 - accuracy: 0.3388 - val_loss: 1.0986 - val_accuracy: 0.3369
Epoch 6/50
1065/1065 [==============================] - 36s 34ms/step - loss: 1.0988 - accuracy: 0.3374 - val_loss: 1.0990 - val_accuracy: 0.3331
Epoch 7/50
1065/1065 [==============================] - 34s 32ms/step - loss: 1.0986 - accuracy: 0.3379 

C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 1/50
1016/1016 [==============================] - 43s 36ms/step - loss: 1.0990 - accuracy: 0.3332 - val_loss: 1.0989 - val_accuracy: 0.3322
Epoch 2/50
1016/1016 [==============================] - 34s 33ms/step - loss: 1.0986 - accuracy: 0.3351 - val_loss: 1.0988 - val_accuracy: 0.3336
Epoch 3/50
1016/1016 [==============================] - 32s 32ms/step - loss: 1.0987 - accuracy: 0.3348 - val_loss: 1.0987 - val_accuracy: 0.3339
Epoch 4/50
1016/1016 [==============================] - 33s 32ms/step - loss: 1.0988 - accuracy: 0.3266 - val_loss: 1.0987 - val_accuracy: 0.3322
Epoch 5/50
1016/1016 [==============================] - 32s 32ms/step - loss: 1.0987 - accuracy: 0.3318 - val_loss: 1.0987 - val_accuracy: 0.3319
Epoch 6/50
1016/1016 [==============================] - 32s 32ms/step - loss: 1.0987 - accuracy: 0.3345 - val_loss: 1.0988 - val_accuracy: 0.3313
Epoch 7/50
1016/1016 [==============================] - 32s 32ms/step - loss: 1.0987 - accuracy: 0.3343 - val_loss: 1.0989 

C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 1/50
1034/1034 [==============================] - 40s 32ms/step - loss: 1.0988 - accuracy: 0.3373 - val_loss: 1.0989 - val_accuracy: 0.3282
Epoch 2/50
1034/1034 [==============================] - 29s 28ms/step - loss: 1.0990 - accuracy: 0.3335 - val_loss: 1.0989 - val_accuracy: 0.3276
Epoch 3/50
1034/1034 [==============================] - 30s 29ms/step - loss: 1.0988 - accuracy: 0.3256 - val_loss: 1.0986 - val_accuracy: 0.3414
Epoch 4/50
1034/1034 [==============================] - 31s 30ms/step - loss: 1.0987 - accuracy: 0.3378 - val_loss: 1.0988 - val_accuracy: 0.3304
Epoch 5/50
1034/1034 [==============================] - 32s 31ms/step - loss: 1.0987 - accuracy: 0.3348 - val_loss: 1.0986 - val_accuracy: 0.3411
Epoch 6/50
1034/1034 [==============================] - 32s 31ms/step - loss: 1.0988 - accuracy: 0.3283 - val_loss: 1.0986 - val_accuracy: 0.3417
Epoch 7/50
1034/1034 [==============================] - 31s 30ms/step - loss: 1.0987 - accuracy: 0.3341 - val_loss: 1.0988 

C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\uabic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50
1008/1008 [==============================] - 46s 38ms/step - loss: 1.0987 - accuracy: 0.3343 - val_loss: 1.0982 - val_accuracy: 0.3473
Epoch 2/50
1008/1008 [==============================] - 35s 34ms/step - loss: 1.0990 - accuracy: 0.3335 - val_loss: 1.0987 - val_accuracy: 0.3184
Epoch 3/50
1008/1008 [==============================] - 35s 35ms/step - loss: 1.0987 - accuracy: 0.3303 - val_loss: 1.0991 - val_accuracy: 0.3184
Epoch 4/50
1008/1008 [==============================] - 34s 34ms/step - loss: 1.0989 - accuracy: 0.3312 - val_loss: 1.0986 - val_accuracy: 0.3482
Epoch 5/50
1008/1008 [==============================] - 35s 35ms/step - loss: 1.0988 - accuracy: 0.3304 - val_loss: 1.0988 - val_accuracy: 0.3184
Epoch 6/50
1008/1008 [==============================] - 34s 34ms/step - loss: 1.0988 - accuracy: 0.3320 - val_loss: 1.0988 - val_accuracy: 0.3482
Epoch 7/50
1008/1008 [==============================] - 34s 34ms/step - loss: 1.0987 - accuracy: 0.3288 - val_loss: 1.0990 -

In [ ]:
##Transformer

In [5]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Directories
processed_matrices_dir = "GSE235063_RAW/processed_matrices_csv"
output_dir = "results/transformer_analysis"
os.makedirs(output_dir, exist_ok=True)

# Transformer Model Definition
def build_transformer_model(input_shape, num_heads, num_classes):
    inputs = Input(shape=input_shape)

    # Multi-Head Attention Layer
    attention_layer = MultiHeadAttention(num_heads=num_heads, key_dim=input_shape[-1])
    attention_output = attention_layer(inputs, inputs)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output + inputs)

    # Feedforward Network
    ffn_output = Dense(64, activation="relu")(attention_output)
    ffn_output = Dense(input_shape[-1], activation="linear")(ffn_output)
    ffn_output = LayerNormalization(epsilon=1e-6)(ffn_output + attention_output)

    # Global Average Pooling
    pooled_output = GlobalAveragePooling1D()(ffn_output)

    # Final Classification Layer
    outputs = Dense(num_classes, activation="softmax")(pooled_output)

    return Model(inputs, outputs), attention_layer

# Train Transformer and Extract Feature Importance
def train_transformer_and_extract_importance(sequences, labels, gene_names, patient):
    # Normalize data
    scaler = MinMaxScaler()
    sequences = scaler.fit_transform(sequences)

    # Reshape to (samples, timesteps, features)
    sequences = sequences.reshape((sequences.shape[0], sequences.shape[1], 1))

    # Generate dummy labels matching samples (one per row/gene)
    labels = np.random.randint(0, 3, size=sequences.shape[0])  # Placeholder for actual labels

    # 70:15:15 split (train, validation, test)
    split_1 = int(0.7 * len(sequences))
    split_2 = int(0.85 * len(sequences))
    X_train, X_val, X_test = (
        sequences[:split_1],
        sequences[split_1:split_2],
        sequences[split_2:],
    )
    y_train, y_val, y_test = labels[:split_1], labels[split_1:split_2], labels[split_2:]

    # Build Transformer model
    model, attention_layer = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]), num_heads=4, num_classes=3
    )
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=16,
        verbose=1
    )

    # Save training metrics
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(output_dir, f"{patient}_training_metrics.csv"), index=False)

    # Evaluate on test data
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=["DX", "REL", "REM"])
    confusion = confusion_matrix(y_test, y_pred)

    # Save evaluation metrics
    with open(os.path.join(output_dir, f"{patient}_evaluation_metrics.txt"), "w") as f:
        f.write(f"Accuracy: {accuracy}\n")
        f.write(f"Classification Report:\n{report}\n")
        f.write(f"Confusion Matrix:\n{confusion}\n")

    # Manually extract attention scores using the attention layer
    attention_model = Model(inputs=model.input, outputs=attention_layer.call(model.input, model.input))
    attention_weights = attention_model.predict(sequences)

    # Aggregate attention scores across all heads and timesteps
    feature_importance = np.mean(np.abs(attention_weights), axis=(1, 2))  # Average across time and heads

    # Rank features by importance
    feature_ranking = pd.DataFrame({
        "Gene": gene_names,
        "Importance": feature_importance
    }).sort_values(by="Importance", ascending=False)

    return feature_ranking

# Process one patient at a time
def process_patient(patient, files):
    print(f"Processing {patient}...")

    # Load DX, REL, REM data
    dx_file = os.path.join(processed_matrices_dir, files.get("DX"))
    rel_file = os.path.join(processed_matrices_dir, files.get("REL"))
    rem_file = os.path.join(processed_matrices_dir, files.get("REM"))

    dx_df = pd.read_csv(dx_file)
    rel_df = pd.read_csv(rel_file)
    rem_df = pd.read_csv(rem_file)

    # Extract genes and samples
    common_genes = set(dx_df["Gene"]).intersection(rel_df["Gene"]).intersection(rem_df["Gene"])
    dx_df = dx_df[dx_df["Gene"].isin(common_genes)].set_index("Gene")
    rel_df = rel_df[rel_df["Gene"].isin(common_genes)].set_index("Gene")
    rem_df = rem_df[rem_df["Gene"].isin(common_genes)].set_index("Gene")

    # Combine DX, REL, REM into sequences (samples x timepoints)
    sequences = np.stack([dx_df.mean(axis=1).values, rel_df.mean(axis=1).values, rem_df.mean(axis=1).values], axis=1)
    labels = np.array([0, 1, 2])  # DX=0, REL=1, REM=2

    # Train Transformer and extract feature importance
    feature_ranking = train_transformer_and_extract_importance(sequences, labels, dx_df.index.tolist(), patient)

    # Save top 100 features
    top_100_features = feature_ranking.head(100)
    top_100_features.to_csv(os.path.join(output_dir, f"{patient}_top_100_features.csv"), index=False)

    print(f"Completed {patient}")

# Organize files by patient
file_groups = {}
for file in os.listdir(processed_matrices_dir):
    if not file.endswith("_processed_matrix.csv"):
        continue
    parts = file.split("_")
    patient, state = parts[1], parts[2]
    file_groups.setdefault(patient, {})[state] = file

# Run the analysis
for patient, files in file_groups.items():
    process_patient(patient, files)

print("Transformer feature importance analysis completed.")


In [1]:
pip show tensorflow


Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\uabic\appdata\roaming\python\python39\site-packages
Requires: tensorflow-intel
Required-by: tf_keras
Note: you may need to restart the kernel to use updated packages.


In [ ]:
###DEG statistical + BDM

In [7]:
import os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from pybdm import BDM

# Initialize BDM object
bdm = BDM(ndim=2)

# Define directories
processed_matrices_dir = "GSE235063_RAW/processed_matrices_csv"
output_dir = "results/statistical_deg_bdm"
os.makedirs(output_dir, exist_ok=True)

# Helper: Compute Differential Expression
def compute_deg(dx_df, rel_df, rem_df, logfc_threshold=1.0, pval_threshold=0.05):
    epsilon = 1e-9  # Small constant to avoid log(0)
    genes = dx_df.index
    deg_results = []

    for gene in genes:
        dx_vals = dx_df.loc[gene].values
        rel_vals = rel_df.loc[gene].values
        rem_vals = rem_df.loc[gene].values

        # Compute log fold changes
        logfc_dx_rel = np.log2(np.mean(rel_vals) + epsilon) - np.log2(np.mean(dx_vals) + epsilon)
        logfc_dx_rem = np.log2(np.mean(rem_vals) + epsilon) - np.log2(np.mean(dx_vals) + epsilon)
        logfc_rel_rem = np.log2(np.mean(rem_vals) + epsilon) - np.log2(np.mean(rel_vals) + epsilon)

        # Perform t-tests
        pval_dx_rel = ttest_ind(dx_vals, rel_vals, equal_var=False).pvalue
        pval_dx_rem = ttest_ind(dx_vals, rem_vals, equal_var=False).pvalue
        pval_rel_rem = ttest_ind(rel_vals, rem_vals, equal_var=False).pvalue

        deg_results.append({
            "Gene": gene,
            "LogFC_DX_REL": logfc_dx_rel,
            "LogFC_DX_REM": logfc_dx_rem,
            "LogFC_REL_REM": logfc_rel_rem,
            "Pval_DX_REL": pval_dx_rel,
            "Pval_DX_REM": pval_dx_rem,
            "Pval_REL_REM": pval_rel_rem
        })

    # Create DEG DataFrame
    deg_df = pd.DataFrame(deg_results)

    # Adjust p-values using Benjamini-Hochberg correction
    deg_df["Adj_Pval_DX_REL"] = multipletests(deg_df["Pval_DX_REL"], method="fdr_bh")[1]
    deg_df["Adj_Pval_DX_REM"] = multipletests(deg_df["Pval_DX_REM"], method="fdr_bh")[1]
    deg_df["Adj_Pval_REL_REM"] = multipletests(deg_df["Pval_REL_REM"], method="fdr_bh")[1]

    # Filter significant DEGs
    deg_df = deg_df[
        (deg_df["Adj_Pval_DX_REL"] < pval_threshold) |
        (deg_df["Adj_Pval_DX_REM"] < pval_threshold) | 
        (deg_df["Adj_Pval_REL_REM"] < pval_threshold)
    ]

    # Select top 100 DEGs by absolute logFC
    deg_df["Abs_LogFC"] = deg_df[["LogFC_DX_REL", "LogFC_DX_REM", "LogFC_REL_REM"]].abs().max(axis=1)
    top_100_degs = deg_df.nlargest(100, "Abs_LogFC")

    # Add up/downregulated classification
    top_100_degs["Regulation"] = top_100_degs.apply(
        lambda row: "Upregulated" if row["LogFC_DX_REL"] > 0 else "Downregulated", axis=1
    )

    return top_100_degs

# Updated Helper: Compute BDM Perturbation Analysis
def compute_bdm(deg_df, dx_df, rel_df, rem_df):
    # Align columns (samples) across all DataFrames
    common_samples = dx_df.columns.intersection(rel_df.columns).intersection(rem_df.columns)
    dx_df = dx_df[common_samples]
    rel_df = rel_df[common_samples]
    rem_df = rem_df[common_samples]
    
    bdm_results = []
    for gene in deg_df["Gene"]:
        dx_vals = dx_df.loc[gene].values
        rel_vals = rel_df.loc[gene].values
        rem_vals = rem_df.loc[gene].values

        # Ensure dimensions match
        if len(dx_vals) != len(rel_vals) or len(dx_vals) != len(rem_vals):
            print(f"Skipping gene {gene} due to mismatched dimensions.")
            continue

        # Compute binary matrices
        binary_dx = (dx_vals > np.median(dx_vals)).astype(int)
        binary_rel = (rel_vals > np.median(rel_vals)).astype(int)
        binary_rem = (rem_vals > np.median(rem_vals)).astype(int)

        # Compute BDM for state transitions
        try:
            bdm_dx_rel = bdm.bdm(np.vstack([binary_dx, binary_rel]))
            bdm_rel_rem = bdm.bdm(np.vstack([binary_rel, binary_rem]))
            bdm_dx_rem = bdm.bdm(np.vstack([binary_dx, binary_rem]))

            bdm_results.append({
                "Gene": gene,
                "BDM_DX_REL": bdm_dx_rel,
                "BDM_REL_REM": bdm_rel_rem,
                "BDM_DX_REM": bdm_dx_rem
            })
        except ValueError as e:
            print(f"Error processing gene {gene}: {e}")
            continue

    return pd.DataFrame(bdm_results)

# Rest of the workflow remains the same.
# Main Workflow
def process_patient(patient, files):
    print(f"Processing {patient}...")

    dx_file = os.path.join(processed_matrices_dir, files["DX"])
    rel_file = os.path.join(processed_matrices_dir, files["REL"])
    rem_file = os.path.join(processed_matrices_dir, files["REM"])

    # Load the data
    dx_df = pd.read_csv(dx_file).set_index("Gene")
    rel_df = pd.read_csv(rel_file).set_index("Gene")
    rem_df = pd.read_csv(rem_file).set_index("Gene")

    # Filter out all-zero genes
    dx_df = dx_df[(dx_df.T != 0).any()]
    rel_df = rel_df[(rel_df.T != 0).any()]
    rem_df = rem_df[(rem_df.T != 0).any()]

    # Align indices of all DataFrames (genes)
    common_genes = dx_df.index.intersection(rel_df.index).intersection(rem_df.index)
    dx_df = dx_df.loc[common_genes]
    rel_df = rel_df.loc[common_genes]
    rem_df = rem_df.loc[common_genes]

    # Compute DEGs
    top_100_degs = compute_deg(dx_df, rel_df, rem_df)
    top_100_degs.to_csv(os.path.join(output_dir, f"{patient}_top_100_degs.csv"), index=False)

    # Compute BDM values
    bdm_results = compute_bdm(top_100_degs, dx_df, rel_df, rem_df)
    bdm_results.to_csv(os.path.join(output_dir, f"{patient}_bdm_results.csv"), index=False)

# Organize files by patient
file_groups = {}
for file in os.listdir(processed_matrices_dir):
    if not file.endswith("_processed_matrix.csv"):
        continue
    parts = file.split("_")
    patient, state = parts[1], parts[2]
    file_groups.setdefault(patient, {})[state] = file

# Run the analysis
for patient, files in file_groups.items():
    process_patient(patient, files)

print("Statistical DEG and BDM analysis completed.")


Processing AML16...
Error processing gene PRTN3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene CD24: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene EPB41L3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene BIRC7: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene POU4F1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene ELANE: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene C15orf48: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene CAV1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene OLR1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene PPBP: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene MPO: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene NPW: Compute

Processing AML6...
Error processing gene HBA1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBA2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene ALAS2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBM: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene AHSP: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene GYPA: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HEMGN: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBB: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene MYO6: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene G0S2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene SPAG6: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene CD14: Computed BDM is 0

Processing AML2...
Error processing gene PLBD1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene AC245128.3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene SERPINB2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene MINDY4B: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene DNTT: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene CD14: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene SERPINA1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene S100A12: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene TLR8: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene S100A8: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene DLX5: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene M

Processing AML15...
Error processing gene IL5RA: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene TCN1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene PRAME: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene POU4F1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HOXA9: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene EPB41L3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene SLC22A15: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene TBX1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene ENHO: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene PF4: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene NEGR1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene CLEC10A: Co

Processing AML7...
Error processing gene CXCL3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBA2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene FOXC1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBA1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene EMX2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene PROK2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene CXCL2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene CXADR: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBM: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene NIPAL4: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene LIF: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene ITGB8: Computed BDM

...

Error processing gene HBG2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene MPO: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBA1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBA2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene IGHG3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene ALAS2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBM: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBG1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene IGLC3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene IGHA1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene TPPP3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene CXCL8: Computed BDM is 0, dataset may ha

Processing AML22...
Error processing gene HBG1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene SERPINA1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HBG2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene FCN1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HOXB-AS3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene IGLON5: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene S100A12: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene EPB41L3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene PRAME: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HOXB3: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene LGALS2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene TGF

Processing AML21...
Error processing gene ZNF521: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene FCGR3A: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene XIST: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene AP003498.1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene RRAD: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene GLI2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene AC007384.1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene AL161772.1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene SIPA1L2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene FPR1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene LGR4: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gen

Processing AML24...
Error processing gene RPS4Y1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene EIF1AY: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene DDX3Y: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene IGHG4: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene MRC1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene ENPP2: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene HRK: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene IGHG1: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene SLC25A21: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene PKLR: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene PKIB: Computed BDM is 0, dataset may have incorrect dimensions
Error processing gene PF4: Compute

Statistical DEG and BDM analysis completed.


In [ ]:
##BDM analysis on identified DEGs

In [10]:
import os
import pandas as pd
import numpy as np
from pybdm import BDM
from scipy.stats import spearmanr

# Initialize BDM object
bdm = BDM(ndim=2)

# Define directories
processed_matrices_dir = "GSE235063_RAW/processed_matrices_csv"
top_degs_dir = "results/statistical_deg_bdm"
bdm_results_dir = "results/bdm_analysis"
os.makedirs(bdm_results_dir, exist_ok=True)

# Thresholds for binarization
thresholds = [0.1, 0.5]

# Helper: Compute Spearman adjacency matrix
def compute_spearman_adjacency(data):
    adjacency = np.zeros((data.shape[0], data.shape[0]))
    for i in range(data.shape[0]):
        for j in range(i + 1, data.shape[0]):  # Avoid redundant computation
            corr, _ = spearmanr(data[i, :], data[j, :])
            adjacency[i, j] = corr
            adjacency[j, i] = corr  # Symmetric matrix
    return adjacency

# Helper: Perform BDM Perturbation Analysis
def bdm_analysis(matrix, threshold):
    binary_matrix = (matrix > threshold).astype(int)
    original_bdm = bdm.bdm(binary_matrix)

    # Node perturbation (set each node to 0)
    perturbed_bdm_values = []
    for i in range(binary_matrix.shape[0]):
        perturbed_matrix = binary_matrix.copy()
        perturbed_matrix[i, :] = 0
        perturbed_matrix[:, i] = 0
        perturbed_bdm_values.append(original_bdm - bdm.bdm(perturbed_matrix))

    return original_bdm, perturbed_bdm_values

# Process each patient's DEGs and compute BDM
for patient_file in os.listdir(top_degs_dir):
    if not patient_file.endswith("_top_100_degs.csv"):
        continue

    patient_name = patient_file.split("_top_100_degs.csv")[0]
    print(f"Processing patient: {patient_name}")

    # Create a directory for the patient
    patient_bdm_dir = os.path.join(bdm_results_dir, patient_name)
    os.makedirs(patient_bdm_dir, exist_ok=True)

    # Load top 100 DEGs
    top_degs = pd.read_csv(os.path.join(top_degs_dir, patient_file))["Gene"].values

    # Load DX, REL, and REM data
    try:
        dx_file = next(f for f in os.listdir(processed_matrices_dir) if f"{patient_name}_DX" in f)
        rel_file = next(f for f in os.listdir(processed_matrices_dir) if f"{patient_name}_REL" in f)
        rem_file = next(f for f in os.listdir(processed_matrices_dir) if f"{patient_name}_REM" in f)
    except StopIteration:
        print(f"Missing files for patient {patient_name}. Skipping...")
        continue

    dx_df = pd.read_csv(os.path.join(processed_matrices_dir, dx_file)).set_index("Gene").loc[top_degs]
    rel_df = pd.read_csv(os.path.join(processed_matrices_dir, rel_file)).set_index("Gene").loc[top_degs]
    rem_df = pd.read_csv(os.path.join(processed_matrices_dir, rem_file)).set_index("Gene").loc[top_degs]

    # Concatenate counts
    combined_matrix = np.hstack([dx_df.values, rel_df.values, rem_df.values])

    # Compute adjacency matrix
    adjacency_matrix = compute_spearman_adjacency(combined_matrix)

    # Perform BDM analysis for each threshold
    for threshold in thresholds:
        original_bdm, perturbed_bdm_values = bdm_analysis(adjacency_matrix, threshold)

        # Save results
        result_path = os.path.join(patient_bdm_dir, f"{patient_name}_bdm_threshold_{threshold}.csv")
        pd.DataFrame({
            "Gene": top_degs,
            "Perturbed_BDM": perturbed_bdm_values
        }).to_csv(result_path, index=False)

print("BDM analysis completed and results saved.")


Processing patient: AML10
Processing patient: AML11
Processing patient: AML15
Processing patient: AML16
Processing patient: AML21
Processing patient: AML22
Processing patient: AML24
Processing patient: AML27
Processing patient: AML2
Processing patient: AML5
Processing patient: AML6
Processing patient: AML7
Processing patient: AML8
Processing patient: AML9
BDM analysis completed and results saved.


In [ ]:
##NNMF: transcriptional modules; hierarchical clustering

In [16]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from scipy.stats import spearmanr
from pybdm import BDM

# Initialize BDM object
bdm = BDM(ndim=2)

# Directories
processed_matrices_dir = "GSE235063_RAW/processed_matrices_csv"
nnmf_results_dir = "results/nnmf_analysis"
os.makedirs(nnmf_results_dir, exist_ok=True)

# Parameters
n_components = 5  # Number of transcriptional modules
top_genes_per_module = 50  # Top genes to keep per module for BDM analysis
thresholds = [0.1, 0.5]  # Thresholds for binarization

# Helper: Compute Spearman adjacency matrix
def compute_spearman_adjacency(data):
    adjacency = np.zeros((data.shape[0], data.shape[0]))
    for i in range(data.shape[0]):
        for j in range(i + 1, data.shape[0]):  # Avoid redundant computation
            corr, _ = spearmanr(data[i, :], data[j, :])
            adjacency[i, j] = corr
            adjacency[j, i] = corr  # Symmetric matrix
    return adjacency

# Helper: Perform BDM Perturbation Analysis
def bdm_analysis(matrix, threshold):
    binary_matrix = (matrix > threshold).astype(int)
    original_bdm = bdm.bdm(binary_matrix)

    # Node perturbation (set each node to 0)
    perturbed_bdm_values = []
    for i in range(binary_matrix.shape[0]):
        perturbed_matrix = binary_matrix.copy()
        perturbed_matrix[i, :] = 0
        perturbed_matrix[:, i] = 0
        perturbed_bdm_values.append(original_bdm - bdm.bdm(perturbed_matrix))

    return perturbed_bdm_values

# Helper: Apply NNMF
def apply_nnmf(expression_matrix, n_components):
    model = NMF(n_components=n_components, init="random", random_state=42, max_iter=1000)
    W = model.fit_transform(expression_matrix)
    H = model.components_
    return W, H

# Process each patient's data
patients = set([file.split("_")[1] for file in os.listdir(processed_matrices_dir) if file.endswith("_processed_matrix.csv")])

for patient_id in patients:
    print(f"Processing patient: {patient_id}")

    # Create directories
    patient_nnmf_dir = os.path.join(nnmf_results_dir, patient_id)
    patient_bdm_dir = os.path.join(patient_nnmf_dir, "bdm_results")
    os.makedirs(patient_nnmf_dir, exist_ok=True)
    os.makedirs(patient_bdm_dir, exist_ok=True)

    # Construct file paths and check existence
    try:
        dx_file = next(f for f in os.listdir(processed_matrices_dir) if f"_{patient_id}_DX_processed_matrix.csv" in f)
        rel_file = next(f for f in os.listdir(processed_matrices_dir) if f"_{patient_id}_REL_processed_matrix.csv" in f)
        rem_file = next(f for f in os.listdir(processed_matrices_dir) if f"_{patient_id}_REM_processed_matrix.csv" in f)
    except StopIteration:
        print(f"Missing one or more files for patient {patient_id}. Skipping...")
        continue

    dx_df = pd.read_csv(os.path.join(processed_matrices_dir, dx_file)).set_index("Gene")
    rel_df = pd.read_csv(os.path.join(processed_matrices_dir, rel_file)).set_index("Gene")
    rem_df = pd.read_csv(os.path.join(processed_matrices_dir, rem_file)).set_index("Gene")

    # Concatenate expression data
    combined_matrix = pd.concat([dx_df, rel_df, rem_df], axis=1).values
    gene_names = dx_df.index.values

    # Apply NNMF
    W, H = apply_nnmf(combined_matrix, n_components)

    # Identify top genes per module
    nnmf_results = []
    for module_idx in range(W.shape[1]):
        module_scores = W[:, module_idx]
        ranked_genes = np.argsort(module_scores)[::-1]  # Sort in descending order
        top_genes = ranked_genes[:top_genes_per_module]
        for gene_idx in top_genes:
            nnmf_results.append({
                "Module": module_idx + 1,
                "Gene": gene_names[gene_idx],
                "Score": module_scores[gene_idx]
            })

    nnmf_results_df = pd.DataFrame(nnmf_results)
    nnmf_results_df.to_csv(os.path.join(patient_nnmf_dir, f"{patient_id}_nnmf_top_genes.csv"), index=False)

    # Get unique top genes across modules for BDM
    unique_top_genes = nnmf_results_df["Gene"].unique()
    top_gene_matrix = pd.DataFrame(combined_matrix, index=gene_names).loc[unique_top_genes].values

    # Compute adjacency matrix for BDM
    adjacency_matrix = compute_spearman_adjacency(top_gene_matrix)

    # Perform BDM analysis for each threshold
    for threshold in thresholds:
        perturbed_bdm_values = bdm_analysis(adjacency_matrix, threshold)
        result_path = os.path.join(patient_bdm_dir, f"{patient_id}_bdm_threshold_{threshold}.csv")
        pd.DataFrame({
            "Gene": unique_top_genes,
            "Perturbed_BDM": perturbed_bdm_values
        }).to_csv(result_path, index=False)

print("NNMF and BDM analysis completed for all patients.")


Processing patient: AML11
Processing patient: AML22
Processing patient: AML27
Processing patient: AML21
Processing patient: AML7
Processing patient: AML16
Processing patient: AML9
Processing patient: AML10
Processing patient: AML6
Processing patient: AML8


C:\Users\uabic\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn(


Processing patient: AML15
Processing patient: AML24
Processing patient: AML5
Processing patient: AML2
NNMF and BDM analysis completed for all patients.


In [ ]:
###Bayesian Ridge Regression 

In [8]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import MinMaxScaler

# Directories
processed_matrices_dir = "GSE235063_RAW/processed_matrices_csv"
ridge_folder = "Bayesian_regression/Bayesian_Ridge"
os.makedirs(ridge_folder, exist_ok=True)

# Helper Function: Load and Process Patient Data
def load_patient_data(files):
    dx_file = files.get("DX")
    rel_file = files.get("REL")
    rem_file = files.get("REM")
    
    if not dx_file or not rel_file or not rem_file:
        raise ValueError("Missing one or more files (DX, REL, REM) for patient.")

    dx_df = pd.read_csv(os.path.join(processed_matrices_dir, dx_file))
    rel_df = pd.read_csv(os.path.join(processed_matrices_dir, rel_file))
    rem_df = pd.read_csv(os.path.join(processed_matrices_dir, rem_file))

    common_genes = set(dx_df["Gene"]).intersection(rel_df["Gene"]).intersection(rem_df["Gene"])
    dx_df = dx_df[dx_df["Gene"].isin(common_genes)].set_index("Gene")
    rel_df = rel_df[rel_df["Gene"].isin(common_genes)].set_index("Gene")
    rem_df = rem_df[rem_df["Gene"].isin(common_genes)].set_index("Gene")

    combined_data = pd.concat([dx_df.mean(axis=1), rel_df.mean(axis=1), rem_df.mean(axis=1)], axis=1)
    combined_data.columns = ["DX", "REL", "REM"]

    return combined_data

# Bayesian Ridge Regression
def bayesian_ridge_regression(patient, files):
    print(f"Processing Bayesian Ridge Regression for patient: {patient}...")
    data = load_patient_data(files)

    X = data.values.T  # Transpose to treat genes as samples
    y = np.arange(X.shape[0])  # Dummy labels for each gene

    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    model = BayesianRidge()
    model.fit(X, y)

    feature_importance = pd.DataFrame({
        "Gene": data.index,
        "Importance": np.abs(model.coef_)
    }).sort_values(by="Importance", ascending=False)

    output_file = os.path.join(ridge_folder, f"{patient}_ridge_importance.csv")
    feature_importance.head(30).to_csv(output_file, index=False)  # Save top 30 features
    print(f"Saved Bayesian Ridge results for patient: {patient}")

# Organize files by patient
file_groups = {}
for file in os.listdir(processed_matrices_dir):
    if not file.endswith("_processed_matrix.csv"):
        continue
    parts = file.split("_")
    patient, state = parts[1], parts[2]
    file_groups.setdefault(patient, {})[state] = file

# Run Analysis for All Patients
for patient, files in file_groups.items():
    try:
        bayesian_ridge_regression(patient, files)
    except Exception as e:
        print(f"Error processing patient {patient}: {e}")

print("Bayesian Ridge Regression completed.")


Processing Bayesian Ridge Regression for patient: AML16...
Saved Bayesian Ridge results for patient: AML16
Processing Bayesian Ridge Regression for patient: AML6...
Saved Bayesian Ridge results for patient: AML6
Processing Bayesian Ridge Regression for patient: AML2...
Saved Bayesian Ridge results for patient: AML2
Processing Bayesian Ridge Regression for patient: AML15...
Saved Bayesian Ridge results for patient: AML15
Processing Bayesian Ridge Regression for patient: AML7...
Saved Bayesian Ridge results for patient: AML7
Processing Bayesian Ridge Regression for patient: AML8...
Saved Bayesian Ridge results for patient: AML8
Processing Bayesian Ridge Regression for patient: AML5...
Saved Bayesian Ridge results for patient: AML5
Processing Bayesian Ridge Regression for patient: AML27...
Saved Bayesian Ridge results for patient: AML27
Processing Bayesian Ridge Regression for patient: AML9...
Saved Bayesian Ridge results for patient: AML9
Processing Bayesian Ridge Regression for patient: